In [ ]:
import pandas as pd

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../data-2023/in/23-138.csv')

In [ ]:
# Rename the column names
df = df.rename(columns={
    'University of Illinois System - Investment Office': 'Name',
    'Unnamed: 1': 'Coupon',
    'Unnamed: 2': 'Maturity Date',
    'Unnamed: 3': 'Quantity',
    'Unnamed: 4': 'Cost Val',
    'Unnamed: 5': 'Market Val'
})

In [ ]:
# Reset the index and drop rows before '9-200100-Neuberger Intermediate'
index_to_reset = df.index[df['Name'] == '9-200100-Neuberger Intermediate'][0]
df.reset_index(drop=True, inplace=True)

# Now your DataFrame should have the index reset, and rows before '9-200100-Neuberger Intermediate' dropped
# because Agency Funds are unimportant to us.
print(df)

In [ ]:
# Asset Types are header rows in Account or Security column 
def is_asset_type(name):
    investment_types = [
        'Asset-Backed Securities',
        'Commercial Mortgage Backed Securities',
        'Corporate Bonds',
        'Municipal Bonds',
        'U.S. Agency Bonds',
        'International Government Bonds',
        'U.S. Treasury Securities',
        'Cash & Cash Equivalents',
    ]
    return name in investment_types

In [ ]:
# Determine the position where you want to insert the 'Asset_Type' column (e.g., after 'Investment_Type')
insert_position = df.columns.get_loc('Name') + 1

# Create a new column 'Asset Type' and fill it with the corresponding row headers
df['Asset Type'] = df['Name'].where(df['Name'].apply(is_asset_type))

# Forward fill the 'Asset Type' column to propagate the labels down to the rows
df['Asset Type'] = df['Asset Type'].ffill()

In [ ]:
# Sanity check: let's export what we currently have a to .csv and check that Asset Type is right 
df.to_csv('../data-2023/out/asset_type_sanity_check.csv')

In [ ]:
# Alright! Let's drop all the rows with null values (this includes header rows for investment data + other fund data)

# Drop any rows with empty or null values
df = df.dropna()

In [ ]:
print(df)

In [ ]:
# Sanity check: let's export what we currently have a to .csv and check that we only have investment data 
df.to_csv('../data-2023/out/investment_data_sanity_check.csv')

In [ ]:
# Next, let's clean unruly typed data. Investments should obey the schema :
# Account or Security (string)
# Coupon (Date format mm/dd/yyyy)
# Maturity Date (double) 
# Quantity (double) 
# Cost Val (double) 
# Market Val (double)

cnt_old_entries = len(df)

# for example, this bond has switched its Maturity Date and Date Coupon (this is actually one of a kind outlier)
print(df.iloc[135])

# Convert 'Maturity Date' to datetime format
df['Maturity Date'] = pd.to_datetime(df['Maturity Date'], errors='coerce')


# Clean commas from numeric column values using regular expressions
numeric_columns = ['Coupon', 'Quantity', 'Cost Val', 'Market Val']

for col in numeric_columns:
    # clean commas to register values as numbers 
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '', regex=True), errors='coerce')

# Now, the unruly typed data becomes NaN or NaT
print(df.iloc[135])

# Drop all unruly typed data 
df = df.dropna()

cnt_new_entries = len(df)

print(f"\nSuccessfully cleaned {cnt_old_entries - cnt_new_entries} unruly typed values.")

In [ ]:
# Sanity check: let's export what we currently have a to .csv and check that all our values are the right type. 
df.to_csv('../data-2023/out/df_data_typed.csv')